In [43]:
from PIL import Image
import numpy as np
import os
import json


def convert_to_train_id(label_array):
    """
    Cityscapes의 원본 레이블을 학습에 사용되는 레이블로 변환합니다.
    """
    mapping = {
        7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5, 19: 6, 20: 7,
        21: 8, 22: 9, 23: 10, 24: 11, 25: 12, 26: 13, 27: 14,
        28: 15, 31: 16, 32: 17, 33: 18
    }
    return np.vectorize(lambda x: mapping.get(x, 255))(label_array)

def label_to_train_id(gt_mask):
    gt = np.array(gt_mask) if isinstance(gt_mask, Image.Image) else np.array(gt_mask)
    gt = convert_to_train_id(gt)
    return gt

def save_results(results, filename):
    """
    결과를 JSON 파일에 저장합니다.
    
    Args:
        results: 저장할 결과 딕셔너리
        filename: 저장할 파일 이름
    """
    
    Result_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), "Result")
    if not os.path.exists(Result_dir):
        os.makedirs(Result_dir)
    file_path = os.path.join(Result_dir, filename)
    
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            # 기존 파일이 리스트 형식이 아닐 경우 리스트로 감싸줍니다.
            existing_results = json.load(f)
            if not isinstance(existing_results, list):
                existing_results = [existing_results]
    else:
        existing_results = []
    
    existing_results.append(results)
    
    with open(file_path, "w") as f:
        json.dump(existing_results, f, ensure_ascii=False, indent=4)
    print(f"결과가 '{file_path}' 파일에 추가 저장되었습니다.")

import numpy as np

def compute_metrics(eval_pred, metric, num_labels):
    pred, labels = eval_pred
    
    # pred가 이미 최종 예측 결과인 경우 (클래스 인덱스)
    # 메트릭 계산
    metrics = metric.compute(
        predictions=pred,
        references=labels,
        num_labels=num_labels,
        ignore_index=255,
        reduce_labels=False,
    )
    
    return metrics

In [44]:
import torch
import numpy as np
from transformers import OneFormerProcessor, OneFormerForUniversalSegmentation
from torchvision.datasets import Cityscapes
from tqdm import tqdm
import evaluate

# 설정값을 클래스로 정의하여 관리
class AttackConfig:
    model_name = "shi-labs/oneformer_cityscapes_swin_large"
    data = "cityscapes"
    DataSize = 500
    batch_size = 10
    Dataset = "val"

def infer_full_image(image, processor, model, device, split_size=(512, 1024)):
    """
    전체 이미지에 대한 추론을 수행합니다.
    
    Args:
        image: 입력 이미지 (PIL Image)
        processor: 이미지 전처리기
        model: 세그멘테이션 모델
        device: 연산 장치 (CPU/GPU)
        
    Returns:
        numpy array: 세그멘테이션 결과 (height, width)
    """
    # 원본 이미지 크기 확인
    width, height = image.size
    split_height, split_width = split_size
    
    # 결과를 저장할 배열 초기화
    result = np.zeros((height, width), dtype=np.int64)
    
    # 이미지를 4등분하여 처리
    for y in range(0, height, split_height):
        for x in range(0, width, split_width):
            # 이미지 조각 추출
            x_end = min(x + split_width, width)
            y_end = min(y + split_height, height)
            
            # 이미지 조각 생성
            tile = image.crop((x, y, x_end, y_end))
            
            # 모델 추론
            inputs = processor(images=tile, task_inputs=["semantic"], return_tensors="pt").to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                
            # 후처리
            tile_result = processor.post_process_semantic_segmentation(
                outputs, target_sizes=[(tile.size[1], tile.size[0])])[0]
            
            # 결과를 numpy 배열로 변환
            tile_result = tile_result.cpu().numpy().astype(np.int64)
            
            # 결과 배열에 조각 결과 삽입
            result[y:y_end, x:x_end] = tile_result
    
    return result

In [45]:
config = AttackConfig()

# Cityscapes 데이터셋 (fine annotation) 사용
dataset = Cityscapes(root=f"./workspace/DataSet/{config.data}/", split=config.Dataset, mode="fine", target_type="semantic")
selected_indices = list(range(len(dataset)))

# 모델 로드
processor = OneFormerProcessor.from_pretrained(config.model_name)
model = OneFormerForUniversalSegmentation.from_pretrained(config.model_name)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"사용 중인 디바이스: {device}")

# 평가 준비
num_classes = 19
matrix = evaluate.load("mean_iou")
miou_metrics_list = []
# 배치 처리 및 평가
for batch_start in tqdm(range(0, len(selected_indices), config.batch_size), desc="batch"):
    batch_indices = selected_indices[batch_start:batch_start+config.batch_size]
    for idx in tqdm(batch_indices, desc="image", leave=False):
        image, gt_mask = dataset[idx]
        pred = infer_full_image(image, processor, model, device)
        
        # GT 마스크 전처리
        gt = label_to_train_id(gt_mask)
        metrics = compute_metrics([pred, gt], matrix, num_classes)
        miou_metrics_list.append(metrics["mean_iou"])

/opt/conda/lib/python3.11/site-packages/transformers/utils/deprecation.py:172: UserWarning: The following named arguments are not valid for `OneFormerImageProcessor.__init__` and were ignored: '_max_size'
  return func(*args, **kwargs)


사용 중인 디바이스: cuda


batch:   0%|          | 0/50 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/datasets/features/image.py:348: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
batch: 100%|██████████| 50/50 [2:45:48<00:00, 198.98s/it]  


In [46]:
miou_metrics = np.array(miou_metrics_list)
print(np.mean(miou_metrics))

0.6742265417813148


In [49]:
# miou 값이 70% 이상인 수 계산
miou_above_70 = np.sum(np.array(miou_metrics_list) >= 0.8)
print(f"miou 값이 70% 이상인 이미지 수: {miou_above_70}")
print(f"전체 이미지 중 비율: {miou_above_70 / len(miou_metrics_list) * 100:.2f}%")


miou 값이 70% 이상인 이미지 수: 39
전체 이미지 중 비율: 7.80%


In [ ]:
import h5py
import matplotlib.pyplot as plt
import os
import sys
project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), "./workspace/"))
sys.path.append(project_root)


# 클래스별 평균 IoU를 시각화
with h5py.File('segmentation_results.h5', 'r') as f:
    class_stats = f['class_statistics']
    
    class_indices = []
    iou_values = []
    
    for class_idx in range(19):
        class_key = str(class_idx)
        if class_key in class_stats:
            class_indices.append(class_idx)
            iou_values.append(class_stats[class_key].attrs['avg_iou'])
    
    # 막대 그래프로 시각화
    plt.figure(figsize=(12, 6))
    plt.bar(class_indices, iou_values)
    plt.xlabel('클래스 인덱스')
    plt.ylabel('평균 IoU')
    plt.title('클래스별 평균 IoU')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(class_indices)
    plt.show()
    
    # 픽셀 수 대비 IoU 산점도
    gt_pixels = []
    for class_idx in range(19):
        class_key = str(class_idx)
        if class_key in class_stats:
            gt_pixels.append(class_stats[class_key].attrs['total_gt_pixels'])
    
    plt.figure(figsize=(10, 6))
    plt.scatter(gt_pixels, iou_values)
    plt.xscale('log')  # 픽셀 수는 로그 스케일로 표시
    plt.xlabel('GT 픽셀 수 (로그 스케일)')
    plt.ylabel('평균 IoU')
    plt.title('클래스별 픽셀 수 대비 IoU')
    
    # 데이터 포인트에 클래스 인덱스 표시
    for i, class_idx in enumerate(class_indices):
        plt.annotate(str(class_idx), (gt_pixels[i], iou_values[i]))
    
    plt.grid(True, alpha=0.3)
    plt.show()

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'segmentation_results.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)